## Downloader

### usdt symbols

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from downloader import Downloader
downloader = Downloader()

In [ ]:
await downloader.download_trading_symbols()

In [ ]:
import yaml
with open('configs.yaml', 'r') as file:
    configs = yaml.safe_load(file)
    
last_launch_time = configs['last_launch_time']
exclude_symbols = configs['exclude_symbols']
symbols = downloader.load_trading_symbols(last_launch_time, exclude_symbols)
print(len(symbols))
symbols

### fetch recent klines

In [ ]:
from downloader import Downloader
downloader = Downloader()

In [ ]:
symbols = downloader.load_usdt_symbols()['Symbol'].tolist()
print(len(symbols))
# symbols = ["BTCUSDT", "ETHUSDT", "XRPUSDT", "ADAUSDT", "DOGEUSDT"]
intervals = ['1 month', '1 week', '1 day', '6 hours', '1 hour']

In [ ]:
processed_results, klines_complete, incomplete_klines = await downloader.fetch_recent_klines(symbols, intervals)
print(len(processed_results), klines_complete, len(incomplete_klines))

In [ ]:
# processed_results[('BTCUSDT', '1 hour')]

## Processor

### processing klines

In [ ]:
from processor import Processor
processor = Processor()

In [ ]:
import yaml
with open('configs.yaml', 'r') as file:
    configs = yaml.safe_load(file)
    
# intervals = configs['intervals']
intervals = ["5 minutes"]
processor.create_env_data(intervals=intervals, reprocess=True, recalculate_constants=False, batch_size=None)

In [ ]:
processed_klines, processing_issues = processor.process_recent_klines(processed_results)

In [ ]:
processed_klines.keys()

## Plotter

### plotting raw klines

In [ ]:
from plotter import Plotter
plotter = Plotter()

In [ ]:
symbol = "SHIB1000USDT"
interval = "1 day"
df = plotter.load_data(symbol, interval)

In [ ]:
plotter.plot_price_history_and_stats(df, symbol)

In [ ]:
interval = "1 hour"
plotter.plot_raw_klines(interval)

In [ ]:
plotter.check_raw_klines("5 minutes")

### plotting processed klines

In [ ]:
from plotter import Plotter
plotter = Plotter()

In [ ]:
potential_stablecoins = plotter.identify_potential_stablecoins(threshold=0.05, interval="1 day")

In [ ]:
# intervals = ["1 month", "1 week", "1 day", "6 hours", "1 hour", "15 minutes"]
intervals = ["15 minutes"]

for interval in intervals:
    plotter.plot_efficient_symbol_comparison(
        interval=interval,
        num_symbols=None,
        use_processed=True,
        use_all_symbols=True,
        exclude_symbols=['USDCUSDT'],
        show_all_plots=True,
    )

In [ ]:
plotter.plot_recent_data(processed_klines)

## CryptoEnv

### serial load_klines

In [ ]:
import os
import time
import numpy as np
from crypto_env import CryptoEnv, EnvState
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.1'

# Initialize the CryptoEnv
start_time = time.time()
env = CryptoEnv(train_configs={
    'load_run': None,
    'jax_seed': 27,
    'train_envs': 256,
    'train_val_envs': 32,
    'val_envs': 64,
    'parameters': {'some_architecture': {'action_dim': 3}},
    'architecture': 'some_architecture',
    'cumulative_reward_scaling_x': 1.0,
    'cumulative_reward_scaling_y': 1.0,
})
init_time = time.time() - start_time
print(f"Environment initialization time: {init_time:.2f} seconds")

In [ ]:
env.interval_timedelta

In [ ]:
# Choose a random symbol and timestamp
symbol = np.random.choice(env.train_symbols)
interval = env.configs['intervals'][-1]
timestamps = env.data[interval][symbol][:, -1]
timestamp = np.random.choice(timestamps)

# Create a dummy EnvState for testing
dummy_state = EnvState(env_type='train')
dummy_state.reset_history(env.configs['sequence_length'])

# Warm-up run
_ = env.load_klines(symbol, timestamp, 'val')
_ = env.create_observation(_, dummy_state.history)

In [ ]:
import time
import numpy as np

# Number of iterations for each speed test
n_iterations = 256 + 64 * 21

# Number of times to repeat the speed test
n_repeats = 10

# List to store the total times for each repeat
total_times = []

for _ in range(n_repeats):
    start_time = time.time()

    for _ in range(n_iterations):
        klines = env.load_klines(symbol, timestamp)
        # observation = env.create_observation(klines, dummy_state.history)

    end_time = time.time()
    total_time = end_time - start_time
    total_times.append(total_time)

# Calculate mean and variance
mean_time = np.mean(total_times)
variance_time = np.var(total_times)

print(f"Mean time for {n_iterations} iterations: {mean_time:.6f} seconds")
print(f"Variance of time: {variance_time:.6f} seconds^2")
print(f"Average iterations per second: {n_iterations/mean_time:.2f}")

# Optional: Check the shape of the returned observation
# print(f"Shape of returned observation: {observation.shape}")

# If you want to see all the individual times:
for i, t in enumerate(total_times):
    print(f"Run {i+1}: {t:.6f} seconds")

### parallel load_klines

In [ ]:
import os
import time
from crypto_env_jax import CryptoEnvJax
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.1'

# Initialize the environment
start_time = time.time()
env_jax = CryptoEnvJax(train_configs={
    'load_run': None,
    'jax_seed': 42,
    'train_envs': 256,
    'train_val_envs': 32,
    'val_envs': 64,
    'parameters': {'some_architecture': {'action_dim': 3}},
    'architecture': 'some_architecture'
})
init_time = time.time() - start_time
print(f"Environment initialization time: {init_time:.2f} seconds")

In [ ]:
env_jax.data.shape

In [ ]:
import time
import random

# Choose a random symbol
symbol = random.choice(env_jax.train_symbols)
symbol_index = env_jax.symbol_to_index[symbol]

# Get the timestamp range for the chosen symbol
shortest_interval_index = env_jax.interval_to_index[env_jax.configs['intervals'][-1]]
timestamps = env_jax.data[shortest_interval_index, symbol_index, :, 7]
valid_timestamps = timestamps[timestamps != 0]

# Choose a random timestamp within the valid range
timestamp = random.choice(valid_timestamps)

start_time = time.time()
klines = env_jax.load_klines([symbol], [timestamp])
end_time = time.time()

print(f"Random symbol: {symbol}")
print(f"Random timestamp: {timestamp}")
print(f"Time taken to load klines: {end_time - start_time:.6f} seconds")
print(f"Klines shape: {klines.shape}")

In [ ]:
import time
import random
import numpy as np

# Number of symbols and timestamps to test
num_samples = 256 + 64 * 21 * 1

# Choose random symbols
symbols = random.choices(env_jax.train_symbols, k=num_samples)

# Initialize list to store timestamps
timestamps = []

for symbol in symbols:
    symbol_index = env_jax.symbol_to_index[symbol]
    
    # Get the timestamp range for the chosen symbol
    shortest_interval_index = env_jax.interval_to_index[env_jax.configs['intervals'][-1]]
    symbol_timestamps = env_jax.data[shortest_interval_index, symbol_index, :, 7]
    valid_timestamps = symbol_timestamps[symbol_timestamps != 0]
    
    # Choose a random timestamp within the valid range
    timestamps.append(random.choice(valid_timestamps))

# Convert timestamps to a numpy array
timestamps = np.array(timestamps)

# print(f"Random symbols: {symbols}")
# print(f"Random timestamps: {timestamps}")

# Measure time to load multiple klines
start_time = time.time()
klines = env_jax.load_klines(symbols, timestamps)
end_time = time.time()

print(f"Time taken to load {num_samples} klines: {end_time - start_time:.6f} seconds")
print(f"Klines shape: {klines.shape}")

# Calculate average loading time per kline
avg_time_per_kline = (end_time - start_time) / num_samples
print(f"Average time per kline: {avg_time_per_kline:.6f} seconds")

In [ ]:
import time
import random
import numpy as np

# Number of samples to test
num_samples = 256 + 64 * 21

# Choose random symbols and timestamps
symbols = random.choices(env_jax.train_symbols, k=num_samples)
timestamps = []

for symbol in symbols:
    symbol_index = env_jax.symbol_to_index[symbol]
    shortest_interval_index = env_jax.interval_to_index[env_jax.configs['intervals'][-1]]
    symbol_timestamps = env_jax.data[shortest_interval_index, symbol_index, :, 7]
    valid_timestamps = symbol_timestamps[symbol_timestamps != 0]
    timestamps.append(random.choice(valid_timestamps))

timestamps = np.array(timestamps)

# Load klines sequentially
start_time_sequential = time.time()
sequential_klines = []
for symbol, timestamp in zip(symbols, timestamps):
    sequential_klines.append(env_jax.load_klines([symbol], [timestamp])[0])
sequential_klines = np.array(sequential_klines)
end_time_sequential = time.time()

# Load klines all at once
start_time_batch = time.time()
batch_klines = env_jax.load_klines(symbols, timestamps)
end_time_batch = time.time()

# Compare results
are_equal = np.allclose(sequential_klines, batch_klines)
max_diff = np.max(np.abs(sequential_klines - batch_klines))

# Print results
print(f"Number of samples: {num_samples}")
print(f"Sequential loading time: {end_time_sequential - start_time_sequential:.6f} seconds")
print(f"Batch loading time: {end_time_batch - start_time_batch:.6f} seconds")
print(f"Are the results equal? {are_equal}")
print(f"Maximum difference between arrays: {max_diff}")
print(f"Sequential klines shape: {sequential_klines.shape}")
print(f"Batch klines shape: {batch_klines.shape}")

# Calculate and print speedup
speedup = (end_time_sequential - start_time_sequential) / (end_time_batch - start_time_batch)
print(f"Speedup factor: {speedup:.2f}x")

# If results are not equal, print more details
if not are_equal:
    print("\nDetailed comparison:")
    for i, (seq, batch) in enumerate(zip(sequential_klines, batch_klines)):
        if not np.allclose(seq, batch):
            print(f"Mismatch at index {i}:")
            print(f"  Symbol: {symbols[i]}, Timestamp: {timestamps[i]}")
            print(f"  Max difference: {np.max(np.abs(seq - batch))}")
            print(f"  Sequential shape: {seq.shape}, Batch shape: {batch.shape}")
            break

### comparison

In [ ]:
import time
import random
import numpy as np

# Number of samples to test
num_samples = 256 * 1 + 64 * 21 * 1 + 0

# Choose random symbols and timestamps
symbols = random.choices(env_jax.train_symbols, k=num_samples)
timestamps = []

for symbol in symbols:
    symbol_index = env_jax.symbol_to_index[symbol]
    longest_interval_index = env_jax.interval_to_index[env_jax.configs['intervals'][-1]]
    symbol_timestamps = env_jax.data[longest_interval_index, symbol_index, :, 7]
    valid_timestamps = symbol_timestamps[symbol_timestamps != 0]
    timestamps.append(random.choice(valid_timestamps[4:-1]))

timestamps = np.array(timestamps)

# Load klines sequentially using env
start_time_sequential_env = time.time()
sequential_klines_env = []
for symbol, timestamp in zip(symbols, timestamps):
    sequential_klines_env.append(env.load_klines(symbol, timestamp))
sequential_klines_env = np.array(sequential_klines_env)
end_time_sequential_env = time.time()

# Load klines sequentially using env_jax
start_time_sequential_jax = time.time()
sequential_klines_jax = []
for symbol, timestamp in zip(symbols, timestamps):
    sequential_klines_jax.append(env_jax.load_klines([symbol], [timestamp])[0])
sequential_klines_jax = np.array(sequential_klines_jax)
end_time_sequential_jax = time.time()

# Load klines all at once using env_jax
start_time_batch = time.time()
batch_klines = env_jax.load_klines(symbols, timestamps)
end_time_batch = time.time()

# Compare results
are_equal_env_jax = np.allclose(sequential_klines_env, sequential_klines_jax)
are_equal_env_batch = np.allclose(sequential_klines_env, batch_klines)
are_equal_jax_batch = np.allclose(sequential_klines_jax, batch_klines)
max_diff_env_jax = np.max(np.abs(sequential_klines_env - sequential_klines_jax))
max_diff_env_batch = np.max(np.abs(sequential_klines_env - batch_klines))
max_diff_jax_batch = np.max(np.abs(sequential_klines_jax - batch_klines))

# Print results
print(f"Number of samples: {num_samples}")
print(f"Sequential loading time (env): {end_time_sequential_env - start_time_sequential_env:.6f} seconds")
print(f"Sequential loading time (env_jax): {end_time_sequential_jax - start_time_sequential_jax:.6f} seconds")
print(f"Batch loading time (env_jax): {end_time_batch - start_time_batch:.6f} seconds")
print(f"Are env and env_jax sequential results equal? {are_equal_env_jax}")
print(f"Are env sequential and env_jax batch results equal? {are_equal_env_batch}")
print(f"Are env_jax sequential and batch results equal? {are_equal_jax_batch}")
print(f"Maximum difference between env and env_jax sequential: {max_diff_env_jax}")
print(f"Maximum difference between env sequential and env_jax batch: {max_diff_env_batch}")
print(f"Maximum difference between env_jax sequential and batch: {max_diff_jax_batch}")
print(f"Sequential klines shape (env): {sequential_klines_env.shape}")
print(f"Sequential klines shape (env_jax): {sequential_klines_jax.shape}")
print(f"Batch klines shape (env_jax): {batch_klines.shape}")

# Calculate and print speedups
speedup_env_vs_jax = (end_time_sequential_env - start_time_sequential_env) / (end_time_sequential_jax - start_time_sequential_jax)
speedup_env_vs_batch = (end_time_sequential_env - start_time_sequential_env) / (end_time_batch - start_time_batch)
speedup_jax_vs_batch = (end_time_sequential_jax - start_time_sequential_jax) / (end_time_batch - start_time_batch)
print(f"Speedup factor (env_jax sequential vs env): {speedup_env_vs_jax:.2f}x")
print(f"Speedup factor (env_jax batch vs env): {speedup_env_vs_batch:.2f}x")
print(f"Speedup factor (env_jax batch vs env_jax sequential): {speedup_jax_vs_batch:.2f}x")

# If results are not equal, print more details
if not (are_equal_env_jax and are_equal_env_batch and are_equal_jax_batch):
    print("\nDetailed comparison:")
    for i, (env_seq, jax_seq, batch) in enumerate(zip(sequential_klines_env, sequential_klines_jax, batch_klines)):
        if not (np.allclose(env_seq, jax_seq) and np.allclose(env_seq, batch) and np.allclose(jax_seq, batch)):
            print(f"Mismatch at index {i}:")
            print(f"  Symbol: {symbols[i]}, Timestamp: {timestamps[i]}")
            print(f"  Max difference (env vs jax sequential): {np.max(np.abs(env_seq - jax_seq))}")
            print(f"  Max difference (env vs batch): {np.max(np.abs(env_seq - batch))}")
            print(f"  Max difference (jax sequential vs batch): {np.max(np.abs(jax_seq - batch))}")
            print(f"  Env sequential shape: {env_seq.shape}")
            print(f"  Jax sequential shape: {jax_seq.shape}")
            print(f"  Batch shape: {batch.shape}")
            break

In [ ]:
print(sequential_klines_env[0])
print(sequential_klines_env[0].shape)
print(sequential_klines_jax[0])
print(sequential_klines_jax[0].shape)
print(batch_klines[0])
print(batch_klines[0].shape)

### utils init

## Actor

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from actor import Actor
actor = await Actor().initialize()

In [ ]:
processed_klines, klines_complete, incomplete_klines, processing_issues, last_klines = await actor.fetch_and_process_klines(intervals_back=24*30 + 10)
combined_klines = actor.combine_recent_klines(processed_klines)
observations = actor.create_observation(combined_klines)

In [ ]:
processed_klines  # 1726437600000

In [ ]:
actions, action_probs, q_action, v = actor.get_action_values(observations)

In [ ]:
df = actor.create_action_summary(actions, action_probs, q_action, v, last_klines, create_df=True)

In [ ]:
len(df)

In [ ]:
df.head(10)

In [ ]:
from crypto_env import CryptoEnv

train_parameters = {
    "load_run": None,
    "train_frames": 0}

env = CryptoEnv(train_configs=train_parameters)

In [ ]:
import numpy as np

symbol = "BTCUSDT"
timestamp = 1723816800000 + 60 * 60 * 1000
test_klines = env.load_klines(symbol, timestamp)

symbol_state = actor.state.symbol_states[symbol]

test_obs = np.hstack((test_klines, symbol_state['history']['action_history'], symbol_state['history']['cumulative_reward_history']))

actions, action_probs, q_action, v = actor.get_action_values(np.expand_dims(test_obs, axis=0))

print(actions, action_probs, q_action, v)

In [ ]:
test_obs - observations

In [ ]:
test_obs[:,0]

In [ ]:
observations[0,:,0]

In [ ]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
i = np.searchsorted(a, 8.5, side='left')
print(i, a[:i])

In [ ]:
summary = actor.create_action_summary(actions, action_probs, q_action, v, last_klines)
len(summary)

In [ ]:
chosen_symbols = actor.choose_symbols(summary)
len(chosen_symbols)

In [ ]:
chosen_symbols

In [ ]:
chosen_actions = actor.update_trading_symbols(chosen_symbols)
len(chosen_actions)

In [ ]:
chosen_actions

In [ ]:
chosen_actions = await actor.run()
chosen_actions

## Testing Random Stuff

### instruments info

In [ ]:
import aiohttp

base_url = "https://api.bybit.com/v5/market"
category = "linear"

async def get_all_symbols():
    url = f"{base_url}/instruments-info"
    params = {"category": category}
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            data = await response.json()
            if data['retCode'] != 0:
                print(f"Error fetching instrument info: {data['retMsg']}")
                return []
            return [
                {
                        'symbol': symbol['symbol'],
                        # 'tickSize': float(symbol['priceFilter']['tickSize'])
                        # 'all_data': symbol,
                        'qtyStep': symbol['lotSizeFilter']['qtyStep'],
                    }
                    for symbol in data.get('result', {}).get('list', [])
                    if symbol['quoteCoin'] == 'USDT'
                ]

symbols = await get_all_symbols()
len(symbols)

In [ ]:
symbols

### load numpy klines

In [ ]:
import os
import numpy as np
from datetime import datetime


def load_klines_up_to_time(symbol, target_time, n, interval="1 day"):
    # Convert target_time to Unix timestamp if it's a string
    if isinstance(target_time, str):
        target_timestamp = int(datetime.strptime(target_time, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    else:
        target_timestamp = target_time

    # Construct file paths
    data_dir = f"../data/klines_processed/{interval}"  # Adjust this path as needed
    data_file = os.path.join(data_dir, f"{symbol}USDT.npy")
    timestamps_file = os.path.join(data_dir, f"{symbol}USDT_timestamps.npy")
    
    # Load data and timestamps
    data = np.load(data_file)
    timestamps = np.load(timestamps_file)
    
    # Find the index of the closest timestamp <= target_timestamp
    target_index = np.searchsorted(timestamps, target_timestamp, side='right') - 1
    
    # Calculate the start index
    start_index = max(0, target_index - n + 1)
    
    # Return the last n klines up to the target timestamp
    return data[start_index:target_index+1], timestamps[start_index:target_index+1]


# Example usage
symbol = "BTC"
target_time_unix = 1692144000000  # August 16, 2023 00:00:00 UTC
target_time_str = "2024-08-15 23:00:00"
n = 5

# Using Unix timestamp
klines1, timestamps1 = load_klines_up_to_time(symbol, target_time_unix, n)

# Using date-time string
klines2, timestamps2 = load_klines_up_to_time(symbol, target_time_str, n)

print(f"Loaded {len(klines1)} klines for {symbol} using Unix timestamp")
print(f"Loaded {len(klines2)} klines for {symbol} using date-time string")
print("\nLast timestamp (Unix):", timestamps1[-1])
print("Last timestamp (String):", timestamps2[-1])
print("\nLast kline data (Unix):", klines1[-1])
print("Last kline data (String):", klines2[-1])

In [ ]:
klines2

In [ ]:
timestamps1

### kline function

In [ ]:
from data_handler import DataHandler
handler = DataHandler()

In [ ]:
symbol = "BCHUSDT"
interval = "1 day"
length = 3
handler.fetch_kline_data(symbol, interval, length=length)

In [ ]:
response = handler.session.get_kline(
                    category=handler.category,
                    symbol="BCHUSDT",
                    interval=handler.all_intervals["1 hour"],
                    start=1607904000000,
                    end=None,
                    limit=3
                )
len(response['result']['list'])

In [ ]:
response

### unix / datetime

In [ ]:
from datetime import datetime, timezone

def date_to_unix_time(date_str):
    dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    dt = dt.replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)

unix_time = 1725141600000
date = datetime.fromtimestamp(unix_time / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
print(f"{unix_time}  -->  {date}")

date_str = "2023-04-01 00:00:00"
unix_time_back = date_to_unix_time(date_str)
print(f"{date_str}  -->  {unix_time_back}")

### ccxt

In [ ]:
import configs_trader as configs

import ccxt

# Initialize the Bybit exchange
exchange = ccxt.bybit({
    'apiKey': 'configs.API_KEY',
    'secret': 'configs.API_SECRET',
    'enableRateLimit': True,
    'options': {
        'defaultType': 'linear',
        'adjustForTimeDifference': True,
        'timeDifference': 2000
    }
})

In [ ]:
markets = exchange.load_markets()

In [ ]:
# Fetch markets
markets = exchange.fetchMarkets()
len(markets)

In [ ]:
markets[2]

In [ ]:
# Fetch markets
markets = exchange.load_markets()

# Get ticker for a specific symbol
symbol = 'BTC/USDT'
ticker = exchange.fetch_ticker(symbol)

print(f"Current price of {symbol}: {ticker['last']}")

# Fetch account balance
balance = exchange.fetch_balance()
print(f"USDT Balance: {balance['USDT']['free']}")

# Place a market order (be cautious with real accounts)
# order = exchange.create_market_buy_order(symbol, 0.001)  # Buy 0.001 BTC
# print(order)